# Rxiv-Maker: Automated LaTeX Article Generation (Docker-Accelerated)

This notebook allows you to run Rxiv-Maker in Google Colab using Docker containers for **significantly faster setup** and more reliable execution.

**Features:**
- Convert Markdown to LaTeX with academic formatting
- Generate figures from Python scripts and Mermaid diagrams
- Complete LaTeX compilation with bibliography
- Download generated PDF and source files
- **⚡ Fast setup**: ~4 minutes instead of ~20 minutes
- **🔒 Reliable**: Pre-configured environment with all dependencies

**What you'll need:**
- Your manuscript in Markdown format with YAML metadata
- Bibliography file (.bib)
- Figure source files (Python scripts or Mermaid diagrams)

**Technical Note:**     
- This notebook uses `udocker` to run Docker containers in Google Colab with the `rxiv` CLI
- The containers are optimized for AMD64 architecture but work on all platforms via emulation when needed.

## 🐳 Docker Setup and Installation

First, let's install udocker and pull the pre-configured Rxiv-Maker container with all dependencies.

In [ ]:
# @title Install udocker and pull Rxiv-Maker container
print("🐳 Setting up Docker environment...")
print("This takes ~4 minutes instead of ~20 minutes for manual dependency installation.")
print("=" * 70)

# Install udocker (Docker alternative for Google Colab)
print("📦 Installing udocker...")
!pip install udocker -q
!udocker --allow-root install > /dev/null 2>&1

# Pull the pre-configured Rxiv-Maker Docker image
print("📥 Pulling Rxiv-Maker container (henriqueslab/rxiv-maker-base:latest)...")
print("ℹ️ Note: Using latest tag - for specific versions, use henriqueslab/rxiv-maker-base:vX.Y.Z")
print("This container includes:")
print("  - Complete LaTeX distribution (TeX Live)")
print("  - Python 3.11 with scientific libraries")
print("  - R with common packages")
print("  - Node.js 18 with Mermaid CLI")
print("  - All system dependencies")

!udocker --allow-root pull henriqueslab/rxiv-maker-base:latest

# Create a container instance
print("\n🔧 Creating container instance...")
!udocker --allow-root create --name=rxiv-maker henriqueslab/rxiv-maker-base:latest

# Install ezinput for UI components
print("\n📱 Installing UI components...")
!pip install ezinput -q

print("\n✅ Docker environment ready!")
print("Container 'rxiv-maker' created with all dependencies pre-installed.")

## 💾 File Storage Options

**Choose where to store your manuscript files:**

- **📁 Current Colab Session**: Files stored temporarily in this notebook session only. Files are lost when the session ends.
- **☁️ Google Drive**: Files saved to your Google Drive permanently. You can continue working on them later.

**📋 Settings Explanation:**

1. **Store in Google Drive?** 
   - ✅ **Yes**: Files persist between sessions, you can continue work later
   - ❌ **No**: Temporary storage, download files before session ends

2. **Manuscript Name**: Folder name for your project (default: "MANUSCRIPT")

3. **Create new manuscript folder**: Check this if you're using a custom name for the first time

4. **Continue a previous build**: Check this to resume work on files already in Google Drive

In [ ]:
#@title Configure Your Manuscript Settings
import os
from ezinput import EZInput as ezi
from IPython.display import clear_output, display, HTML
from google.colab import output

def print_set(options):
    # Clear previous confirmations
    nLabels = gui._nLabels
    gui[f"label_{nLabels}"].value = "✅ Settings saved successfully!"
    
    # Show summary of selected options
    storage = "Google Drive" if gui["in_gdrive"].value else "Colab Session (temporary)"
    print(f"\n📋 Configuration Summary:")
    print(f"   • Storage location: {storage}")
    print(f"   • Manuscript folder: {gui['man_name'].value}")
    print(f"   • New folder: {'Yes' if gui['new_folder'].value else 'No'}")
    print(f"   • Continue previous: {'Yes' if gui['continue'].value else 'No'}")
    print(f"\n✅ Ready to proceed! Run the next cell to continue.")

gui = ezi()

# Header
gui.add_label("🔧 CONFIGURE YOUR PROJECT SETTINGS")
gui.add_label("=" * 50)

# Storage options
gui.add_label("📍 Storage Location:")
gui.add_check("in_gdrive", description="☁️ Store in Google Drive (persistent)", remember_value=True, value=False)
gui.add_label("   💡 Unchecked = Store in current session only (temporary)")
gui.add_label("")

# Manuscript settings  
gui.add_label("📝 Manuscript Settings:")
gui.add_text_area("man_name", description="Folder name for your manuscript:", remember_value=True, placeholder="MANUSCRIPT", value="MANUSCRIPT")
gui.add_label("")

# Advanced options
gui.add_label("⚙️ Advanced Options:")
gui.add_check("new_folder", description="Create new manuscript folder", remember_value=True, value=False)
gui.add_label("   💡 Check this if using a custom name for the first time")
gui.add_label("")
gui.add_check("continue", description="Continue previous build from Google Drive", remember_value=True, value=False)
gui.add_label("   💡 Check this to resume work on existing files")
gui.add_label("")

# Confirmation button
gui.add_callback("set", print_set, {}, description="💾 Save Settings")
gui.add_label("")
gui.show()

## 📥 Clone Rxiv-Maker Repository

In [ ]:
#@title Clone the Rxiv-Maker repository to access the processing scripts and templates.
import os
import shutil

if gui["in_gdrive"].value:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/rxiv-maker"
    os.environ["MANUSCRIPT_PATH"] = "drive/MyDrive/rxiv-maker" + os.sep + gui["man_name"].value
else:
    PATH = "/content/rxiv-maker"
    os.environ["MANUSCRIPT_PATH"] = "rxiv-maker" + os.sep + gui["man_name"].value

output_path = PATH + os.sep + "output"
output_figures_path = PATH + os.sep + gui["man_name"].value + os.sep + "FIGURES"

if not gui["continue"].value:
  # Clone the repository
  if os.path.exists(PATH):
      shutil.rmtree(PATH)

  !git clone https://github.com/HenriquesLab/rxiv-maker.git {PATH}

  # Create output directory
  !mkdir -p {output_path}
  !mkdir -p {output_figures_path}

  print("✅ Rxiv-Maker repository cloned and set up!")
  print(f"Current directory: {os.getcwd()}")

  if gui["new_folder"].value:
    if os.path.exists(PATH + os.sep + gui["man_name"].value):
        shutil.rmtree(PATH + os.sep + gui["man_name"].value)
    shutil.copytree(PATH + os.sep + "MANUSCRIPT", PATH + os.sep + gui["man_name"].value)
  elif gui["man_name"].value == "EXAMPLE_MANUSCRIPT":
    pass # will use the example manuscript folder
  else:
    os.rename(PATH + os.sep + "MANUSCRIPT", PATH + os.sep + gui["man_name"].value)

# Install rxiv-maker in the container
print("\n📦 Installing Rxiv-Maker in container...")
print("This step installs the rxiv CLI and all Python modules...")

# Install with verbose output and error checking
!udocker --allow-root run -v /content:/workspace rxiv-maker bash -c "cd /workspace{PATH.replace('/content', '')} && pip install -e . && echo '✅ pip install completed'"

# Verify the installation by checking if rxiv command is available
print("\n🔍 Verifying rxiv CLI installation...")
!udocker --allow-root run -v /content:/workspace rxiv-maker bash -c "which rxiv && rxiv --version"

# Test that key Python modules are importable
print("\n🐍 Testing Python module imports...")
!udocker --allow-root run -v /content:/workspace rxiv-maker bash -c "python -c 'from rxiv_maker.cli.main import main; print(\"✅ rxiv_maker module import successful\")'"

print("\n✅ Repository setup and installation completed!")
print("Container now has rxiv CLI available and all modules installed.")

## 📄 Generate PDF Article

In [ ]:
#@title Run setup, generate figures, compile LaTeX and generate PDF
MANUSCRIPT_FOLDER = f"/workspace{PATH.replace('/content', '')}/{gui['man_name'].value}"
MANUSCRIPT_FOLDER_RELATIVE = gui["man_name"].value
!udocker --allow-root run -v /content:/workspace rxiv-maker bash -c "cd /workspace{PATH.replace('/content', '')} && rxiv setup && rxiv clean '{MANUSCRIPT_FOLDER_RELATIVE}' --figures-only && rxiv pdf '{MANUSCRIPT_FOLDER_RELATIVE}'"

## 📍 Important Note About PDF Location

**If you see "PDF generation successful" in the output above but the preview shows "PDF not found":**

    "The Rxiv-Maker Docker container (using the modern `rxiv pdf` CLI) generates the PDF in the manuscript directory (`MANUSCRIPT/`) rather than the `output/` directory. The updated code below will automatically detect the PDF in the correct location.\n",

The build process creates files in multiple locations:
- **LaTeX source files**: Usually in `output/` directory  
- **Final PDF**: Often copied to the manuscript directory for convenience
- **Log files**: May be in either location depending on the build step

The preview and download cells have been updated to check both locations automatically.

## 📱 Preview and Download

In [ ]:
#@title View the generated PDF and download all output files.
import base64
import glob
from IPython.display import HTML, display

# Robust PDF detection - try multiple locations and naming patterns
manuscript_folder = PATH + os.sep + gui["man_name"].value
pdf_locations = [
    # Try output directory first
    os.path.join(output_path, gui["man_name"].value + ".pdf"),
    os.path.join(output_path, "MANUSCRIPT.pdf"),
    # Try manuscript directory
    os.path.join(manuscript_folder, gui["man_name"].value + ".pdf"),
    os.path.join(manuscript_folder, "MANUSCRIPT.pdf"),
    # Try with different naming conventions
    os.path.join(manuscript_folder, f"{gui['man_name'].value.lower()}.pdf"),
    os.path.join(manuscript_folder, "manuscript.pdf"),
]

# Also search for dynamic PDF names (e.g., "2025__author_et_al__rxiv.pdf")
dynamic_pdfs = []
if os.path.exists(manuscript_folder):
    dynamic_pdfs = glob.glob(os.path.join(manuscript_folder, "*__*__*.pdf"))
if os.path.exists(output_path):
    dynamic_pdfs.extend(glob.glob(os.path.join(output_path, "*__*__*.pdf")))

# Combine all potential PDF locations
all_pdf_locations = pdf_locations + dynamic_pdfs

pdf_path = None
pdf_name = None

for path in all_pdf_locations:
    if os.path.exists(path):
        pdf_path = path
        pdf_name = os.path.basename(path)
        print(f"📄 PDF generated successfully using Docker: {pdf_name}")
        print(f"📍 Found at: {path}")
        break

if not pdf_path:
    print("❌ PDF not found. Please check the compilation step above.")
    print("\nSearched locations:")
    for i, path in enumerate(pdf_locations, 1):
        print(f"  {i}. {path}")
    
    if dynamic_pdfs:
        print("\nDynamic PDF patterns searched:")
        for i, path in enumerate(dynamic_pdfs, 1):
            print(f"  {i}. {path}")
    
    print("\nAvailable files in output directory:")
    !ls -la {output_path}
    
    print(f"\nAvailable files in manuscript directory ({manuscript_folder}):")
    !ls -la {manuscript_folder}

if pdf_path:
    print("📄 PDF Preview:")
    print("=" * 30)
    
    # Instead of blocked iframe, use Colab's built-in PDF display
    from google.colab import files
    from IPython.display import display, HTML, Javascript
    import base64
    
    # Method 1: Try Google Colab's built-in PDF viewer
    try:
        # Create a temporary link for preview
        with open(pdf_path, "rb") as f:
            pdf_data = f.read()
            
        # Use a more compatible approach - create download link that can be previewed
        pdf_b64 = base64.b64encode(pdf_data).decode("utf-8")
        
        # Create a download link with preview option
        download_link = f'''
        <div style="border: 2px solid #4CAF50; padding: 15px; margin: 10px 0; border-radius: 5px;">
            <h3>📄 Generated PDF: {pdf_name}</h3>
            <p><strong>File location:</strong> {pdf_path}</p>
            <p><strong>File size:</strong> {len(pdf_data):,} bytes</p>
            
            <div style="margin: 10px 0;">
                <a href="data:application/pdf;base64,{pdf_b64}" 
                   download="{pdf_name}" 
                   style="background: #4CAF50; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; margin-right: 10px;">
                   📥 Download PDF
                </a>
                <a href="data:application/pdf;base64,{pdf_b64}" 
                   target="_blank" 
                   style="background: #2196F3; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px;">
                   👁️ View PDF (opens in new tab)
                </a>
            </div>
            
            <p style="color: #666; font-size: 12px;">
                <strong>Note:</strong> If preview is blocked by your browser, use the download link to save the PDF locally.
                Modern browsers block inline PDF display for security reasons.
            </p>
        </div>
        '''
        
        display(HTML(download_link))
        
        # Also trigger automatic download for user convenience
        print("🎉 PDF generated successfully!")
        print("💡 Click the download button above or the link will download automatically below:")
        
    except Exception as e:
        print(f"⚠️ Could not create preview: {e}")
        print(f"📁 PDF file available at: {pdf_path}")
        
    # Provide direct download via files.download() as backup
    print("\n🔗 Direct download:")
    try:
        files.download(pdf_path)
        print(f"✅ {pdf_name} download initiated")
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print(f"📁 Manual download: Copy from {pdf_path}")

    print("\n" + "=" * 50)
    print("📥 Additional files available below:")
    print("Run the next cells to download LaTeX source and other files.")
    print("=" * 50)

In [ ]:
#@title Download Manuscript Files
import glob
from google.colab import files

# Robust PDF detection - try multiple locations and naming patterns
manuscript_folder = PATH + os.sep + gui["man_name"].value
pdf_locations = [
    # Try output directory first
    os.path.join(output_path, gui["man_name"].value + ".pdf"),
    os.path.join(output_path, "MANUSCRIPT.pdf"),
    # Try manuscript directory
    os.path.join(manuscript_folder, gui["man_name"].value + ".pdf"),
    os.path.join(manuscript_folder, "MANUSCRIPT.pdf"),
    # Try with different naming conventions
    os.path.join(manuscript_folder, f"{gui['man_name'].value.lower()}.pdf"),
    os.path.join(manuscript_folder, "manuscript.pdf"),
]

# Also search for dynamic PDF names (e.g., "2025__author_et_al__rxiv.pdf")
dynamic_pdfs = []
if os.path.exists(manuscript_folder):
    dynamic_pdfs = glob.glob(os.path.join(manuscript_folder, "*__*__*.pdf"))
if os.path.exists(output_path):
    dynamic_pdfs.extend(glob.glob(os.path.join(output_path, "*__*__*.pdf")))

# Combine all potential PDF locations
all_pdf_locations = pdf_locations + dynamic_pdfs

pdf_path = None
pdf_name = None
tex_name = None

for path in all_pdf_locations:
    if os.path.exists(path):
        pdf_path = path
        pdf_name = os.path.basename(path)
        # Determine corresponding TEX file name
        if "MANUSCRIPT" in pdf_name.upper():
            tex_name = "MANUSCRIPT.tex"
        else:
            tex_name = gui["man_name"].value + ".tex"
        break

# Download individual files
print("Click to download individual files:")
print("-" * 40)

# Download PDF
if pdf_path:
    files.download(pdf_path)
    print(f"✅ {pdf_name} downloaded")
else:
    print("❌ PDF not found - check compilation step")

# Download LaTeX source
if tex_name:
    # Try multiple locations for TEX files
    tex_locations = [
        os.path.join(output_path, tex_name),
        os.path.join(manuscript_folder, "output", tex_name),
        os.path.join(manuscript_folder, tex_name)
    ]
    
    tex_found = False
    for tex_path in tex_locations:
        if os.path.exists(tex_path):
            files.download(tex_path)
            print(f"✅ {tex_name} downloaded from {os.path.dirname(tex_path)}")
            tex_found = True
            break
    
    if not tex_found:
        print(f"❌ {tex_name} not found in expected locations")

print("\n📦 Or download everything as ZIP:")

In [ ]:
#@title Download all files as a ZIP
import datetime
import zipfile
import glob

# Create ZIP archive with all output files
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"rxiv_maker_docker_output_{timestamp}.zip"

with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    # Add all files from output directory
    if os.path.exists(output_path):
        for root, dirs, files_list in os.walk(output_path):
            for file in files_list:
                file_path = os.path.join(root, file)
                arcname = os.path.join("output", os.path.relpath(file_path, output_path))
                zipf.write(file_path, arcname)
                print(f"📁 Added to ZIP: {arcname}")
    
    # Add all files from manuscript directory (including generated PDFs, LaTeX files, etc.)
    manuscript_folder = PATH + os.sep + gui["man_name"].value
    if os.path.exists(manuscript_folder):
        # Include all important file types
        important_extensions = ['.pdf', '.tex', '.log', '.aux', '.bbl', '.blg', '.bib']
        
        # Walk through the manuscript directory
        for root, dirs, files_list in os.walk(manuscript_folder):
            for file in files_list:
                # Include important files and all PDFs (including dynamic names)
                if (any(file.lower().endswith(ext) for ext in important_extensions) or 
                    file.lower().endswith('.pdf')):
                    file_path = os.path.join(root, file)
                    if os.path.isfile(file_path):
                        # Create a relative archive path
                        rel_path = os.path.relpath(file_path, manuscript_folder)
                        arcname = os.path.join("manuscript", rel_path)
                        zipf.write(file_path, arcname)
                        print(f"📁 Added to ZIP: {arcname}")

print(f"\n✅ Created ZIP archive: {zip_filename}")
print(f"📦 Archive size: {os.path.getsize(zip_filename)} bytes")

# Download the ZIP file
files.download(zip_filename)
print("🎉 ZIP archive downloaded successfully!")
print("\n🐳 Generated using Docker-accelerated Rxiv-Maker")

## 🔧 Troubleshooting

In [ ]:
#@title If you encounter issues, run this cell to check common problems.
print("🔍 TROUBLESHOOTING CHECKLIST (Docker Version)")
print("=" * 50)

# Check Docker container
print("🐳 Checking Docker container...")
!udocker --allow-root ps

# Check rxiv CLI installation
print("\n⚡ Checking rxiv CLI installation...")
!udocker --allow-root run -v /content:/workspace rxiv-maker bash -c "which rxiv && rxiv --version" || echo "❌ rxiv command not found - installation failed"

# Check Python module imports
print("\n🐍 Checking Python module imports...")
!udocker --allow-root run -v /content:/workspace rxiv-maker bash -c "python -c 'from rxiv_maker.cli.main import main; print(\"✅ Main CLI module import OK\")'" || echo "❌ Cannot import rxiv_maker modules"

# Check required files
print("\n📁 Checking required files...")
required_files = [
    PATH + os.sep + gui["man_name"].value + os.sep + "01_MAIN.md",
    PATH + os.sep + gui["man_name"].value + os.sep + "03_REFERENCES.bib"
]
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file} found")
    else:
        print(f"❌ {file} missing")

# Check source code installation
print("\n🐍 Checking source code installation...")
source_files = [
    PATH + os.sep + "src/rxiv_maker/__init__.py",
    PATH + os.sep + "src/rxiv_maker/cli/main.py",
    PATH + os.sep + "pyproject.toml",
]
for script in source_files:
    if os.path.exists(script):
        print(f"✅ {script} found")
    else:
        print(f"❌ {script} missing")

# Check LaTeX installation in container
print("\n📝 Checking LaTeX in Docker container...")
!udocker --allow-root run rxiv-maker pdflatex --version | head -1

# Check output directory
print("\n📂 Checking output directory...")
if os.path.exists(output_path):
    print("✅ Output directory exists")
    files_in_output = os.listdir(output_path)
    print(f"📁 Files in output: {len(files_in_output)}")
    if files_in_output:
        print(f"   Files: {files_in_output}")
else:
    print("❌ Output directory missing")

# Check manuscript directory for PDF
manuscript_folder = PATH + os.sep + gui["man_name"].value
print(f"\n📂 Checking manuscript directory: {gui['man_name'].value}")
if os.path.exists(manuscript_folder):
    print("✅ Manuscript directory exists")
    files_in_manuscript = os.listdir(manuscript_folder)
    pdf_files = [f for f in files_in_manuscript if f.lower().endswith('.pdf')]
    tex_files = [f for f in files_in_manuscript if f.lower().endswith('.tex')]
    
    if pdf_files:
        print(f"✅ Found PDF files: {pdf_files}")
    else:
        print("❌ No PDF files found in manuscript directory")
    
    if tex_files:
        print(f"✅ Found TEX files: {tex_files}")
    else:
        print("❌ No TEX files found in manuscript directory")
else:
    print("❌ Manuscript directory missing")

# Legacy check for output directory files
if os.path.exists(output_path):
    files_in_output = os.listdir(output_path)
    # Robust PDF and TEX detection for troubleshooting
    primary_pdf_name = gui["man_name"].value + ".pdf"
    fallback_pdf_name = "MANUSCRIPT.pdf"
    primary_tex_name = gui["man_name"].value + ".tex"
    fallback_tex_name = "MANUSCRIPT.tex"
    primary_log_name = gui["man_name"].value + ".log"
    fallback_log_name = "MANUSCRIPT.log"

    # Check TEX files in output
    if primary_tex_name in files_in_output:
        print(f"✅ {primary_tex_name} generated in output")
    elif fallback_tex_name in files_in_output:
        print(f"✅ {fallback_tex_name} generated in output (fallback)")
    else:
        print(f"❌ No TEX file found in output ({primary_tex_name} or {fallback_tex_name})")

    # Check PDF files in output
    if primary_pdf_name in files_in_output:
        print(f"✅ {primary_pdf_name} generated in output")
    elif fallback_pdf_name in files_in_output:
        print(f"✅ {fallback_pdf_name} generated in output (fallback)")
    else:
        print(f"❌ No PDF file found in output ({primary_pdf_name} or {fallback_pdf_name})")

print("\n" + "=" * 50)
print("If you see any ❌ errors above, please:")
print("1. Re-run the setup cells")
print("2. Check that your input files are properly formatted")
print("3. Look at the LaTeX log for compilation errors")
print("4. Verify the Docker container is running properly")
print("5. If rxiv command is missing, re-run the repository cloning cell")

# Show recent LaTeX log if available
log_found = False
log_locations = [
    # Try output directory
    os.path.join(output_path, gui["man_name"].value + ".log"),
    os.path.join(output_path, "MANUSCRIPT.log"),
    # Try manuscript directory
    os.path.join(manuscript_folder, gui["man_name"].value + ".log"),
    os.path.join(manuscript_folder, "MANUSCRIPT.log"),
]

for log_path in log_locations:
    if os.path.exists(log_path):
        log_name = os.path.basename(log_path)
        print(f"\n📄 Recent LaTeX log from {log_name} (last 20 lines):")
        !tail -20 {log_path}
        log_found = True
        break

if not log_found:
    print("\n📄 No LaTeX log files found in output or manuscript directories")

## 📖 Next Steps

**Congratulations!** You've successfully used the Docker-accelerated Rxiv-Maker to generate a publication-ready LaTeX article.

### What you can do next:

1. **Customize your manuscript**: Edit the YAML frontmatter in your Markdown file to change title, authors, affiliations, etc.

2. **Add more figures**: Create Python scripts or Mermaid diagrams in the FIGURES directory

    "3. **Use locally with Docker**: Run `rxiv pdf --engine docker` on your local machine
",

    "4. **Switch to local installation**: Clone the repository and run `rxiv setup && rxiv pdf`
",

### Learn more:
- [Docker Engine Mode Guide](https://github.com/HenriquesLab/rxiv-maker/blob/main/docs/workflows/docker-engine-mode.md)
- [Markdown Guide](https://www.markdownguide.org/)
- [LaTeX Documentation](https://www.latex-project.org/help/documentation/)

### Need help?
- Open an issue on [GitHub](https://github.com/HenriquesLab/rxiv-maker/issues)
- Check the troubleshooting section above
- For Docker-specific issues, see the [Docker Engine Mode Guide](https://github.com/HenriquesLab/rxiv-maker/blob/main/docs/workflows/docker-engine-mode.md)

### Technical Notes:
- This notebook uses `udocker` to run Docker containers in Google Colab
- The `henriqueslab/rxiv-maker-base:latest` image is optimized for AMD64 but works on all platforms
- Container includes: LaTeX, Python 3.11, R, Node.js, and all scientific libraries

---


*This Docker-accelerated notebook was created for the Rxiv-Maker project by the HenriquesLab.**This Docker-accelerated notebook was created for the Rxiv-Maker project by the HenriquesLab.*